# Schnorr with 1-bit challenge and multiple rounds

Using the Fiat-Shamir heuristic to transform an interactive proof with only 1-bit of security per round into a non-interactive protocol is not secure. Here's an implementation of such a thing for the 1-bit challenge Schnorr protocol. Your goal is to break it in the next section

In [ ]:
from common import Transcript, Sha3_256Transcript

In [ ]:
# Secure prime to be used in Schnorr
# 2048-bit MODP Group (https://datatracker.ietf.org/doc/html/rfc3526#section-3)
PRIME = 0xFFFFFFFFFFFFFFFFC90FDAA22168C234C4C6628B80DC1CD129024E088A67CC74020BBEA63B139B22514A08798E3404DDEF9519B3CD3A431B302B0A6DF25F14374FE1356D6D51C245E485B576625E7EC6F44C42E9A637ED6B0BFF5CB6F406B7EDEE386BFB5A899FA5AE9F24117C4B1FE649286651ECE45B3DC2007CB8A163BF0598DA48361C55D39A69163FA8FD24CF5F83655D23DCA3AD961C62F356208552BB9ED529077096966D670C354E4ABC9804F1746C08CA18217C32905E462E36CE3BE39E772C180E86039B2783A2EC07A28FB5C55DF06F4C52C9DE2BCBF6955817183995497CEA956AE515D2261898FA051015728E5A8AACAA68FFFFFFFFFFFFFFFF

# This may take some time since SageMath checks the primality of `PRIME`
F = GF(PRIME)
generator = F(2)

In [ ]:
from tqdm.auto import tqdm

from math import ceil
from random import randint
from dataclasses import dataclass
from typing import List

NUMBER_ITERATIONS = 200


class Party:
    def __init__(self, generator):
        self._generator = F(generator)   


@dataclass
class Proof:
    R_list: List[int]
    s_list: List[int]


class SchnorrProver(Party):
    @staticmethod
    def simulate_send_field_element(element: int, transcript: Transcript):
        """ Appends the element's big endian representation to the transcript """
        prime_byte_length = ceil(len(bin(F.order())[2:]) / 8)
        element_as_bytes = int(element).to_bytes(prime_byte_length, "big")
        transcript.append(element_as_bytes)

    def prove(self, a: int, transcript: Transcript):
        R_list = []
        s_list = []
        for _ in tqdm(range(NUMBER_ITERATIONS)):
            r = randint(0, F.order() - 1)
            R = self._generator ** r
            
            # Fiat-Shamir simulated interactions
            self.simulate_send_field_element(R, transcript)
            b = SchnorrVerifier.simulate_choose_challenge(transcript)
            
            s = (r + a * b) % (F.order() - 1)

            R_list.append(R)
            s_list.append(s)
        
        return Proof(R_list=R_list, s_list=s_list)


In [ ]:

class SchnorrVerifier(Party):
    @staticmethod
    def simulate_choose_challenge(transcript: Transcript):
        """
        Samples a random bit from the transcript
        """
        return int.from_bytes(transcript.sample(), "big") % 2
    
    def verify(self, A: int, proof: Proof, transcript: Transcript):
        for R, s in zip(proof.R_list, proof.s_list):
            # Fiat-Shamir simulated interactions
            SchnorrProver.simulate_send_field_element(R, transcript)
            b = self.simulate_choose_challenge(transcript)
            
            left_hand_side = F(R) * F(A) ** b
            right_hand_side = self._generator ** s

            if left_hand_side != right_hand_side:
                return False
        return True

In [ ]:
generator = F(2)
a = 0xcafe
A = pow(generator, a, PRIME)

# Transcript initialization nonce
nonce = bytes.fromhex("deadbeef") + int(A).to_bytes(2048, "big")

prover = SchnorrProver(generator)
proof = prover.prove(a, Sha3_256Transcript(nonce))

verifier = SchnorrVerifier(generator)
assert(verifier.verify(A, proof, Sha3_256Transcript(nonce)))

# Malicious prover

Your goal here is to come up with a malicious prover that can produce fake proofs and convince a verifier even when he doesn't know the discrete logarithm of the public parameter `A`.

In [ ]:
class SchnorrMaliciousProver(Party):
    def prove(self, transcript: Transcript):
        R_list = []
        s_list = []
        for _ in tqdm(range(NUMBER_ITERATIONS)):
            # COMPLETE
            r = ...
            R = ...

            R_list.append(R)
            s_list.append(s)
        
        return Proof(R_list=R_list, s_list=s_list)

In [ ]:
# [TEST]

generator = F(2)
# a value `A` of which you don't know the discrete logarithm
A = 0x1C6583A8EC6612C6DEAD596A26279C6A7304C1BB9123EBE8B20D2AA978E2C1BF3C7D44533DD05B7B3199452B229301462B9D6DF43700296C306D042313C165E29C916C048A4F618812D4B063EFD497B4C35F2E8020A03D8C351D9C22D7A7F20CF9822AFA4204AAD1F4C1E6D9E71E4F38594C3B16A59E6FED31AAAA848F23AE3A77AAF8C59A74379CBCCBFEA9FD57AD3CA616C0790F4000FA68DE253BEFB13A4A4F0A3D3E321872F1BCA1E205637A9598A231C7B24BFBA2E418434BFAE22E18CE722722B18AC71A650357BDF3175CC2A24C06B416C6C333A1919783E02F2BBDA2BB55E5BFAA42BB5EF541221B28F4ADCECE2A336734D8ED3F084086998384618

# Transcript initialization nonce
nonce = bytes.fromhex("deadbeef") + int(A).to_bytes(2048, "big")

malicious_prover = SchnorrMaliciousProver(generator)
fake_proof = malicious_prover.prove(Sha3_256Transcript(nonce))

verifier = SchnorrVerifier(generator)
assert(verifier.verify(A, fake_proof, Sha3_256Transcript(nonce)))